In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_May7_ntuple_old/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def flat(x,axis=None):
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
# let's define a couple of histograms
# each .Bin defines a new axis of the histogram

hists = {}
hists["Target_epz1_hist"] = hist.Hist("Target SP Kaons",
                                hist.Cat("Ptype", "Type of Particle"),         
                                hist.Bin("pz1", r"$p_z$ [MeV]", 40, 0, 3000),
                                hist.Bin("e1", r"Energy [MeV]", 40, 0, 3000),
                                hist.Bin("theta1", r"$\theta [^{\circ}]$", 40, 0, 90),
                            )

hists["Target_epz2_hist"] = hist.Hist("Target SP Kaons", 
                                hist.Cat("Ptype", "Type of Particle"), 
                                hist.Bin("pz2", r"$p_z$ [MeV]", 40, 0, 3000),  
                                hist.Bin("e2", r"Energy [MeV]", 40, 0, 3000),
                                hist.Bin("theta2", r"$\theta [^{\circ}]$", 40, 0, 90)
                            )


In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    target_kaon_dau1 = ldmx_events['Target_Kaon_dau1']
    target_kaon_dau2 = ldmx_events['Target_Kaon_dau2']
    target_kaon_dau1 = add_angle(target_kaon_dau1)
    target_kaon_dau2 = add_angle(target_kaon_dau2)
    
    masks_id = {
        #'Kshort': 310,
                #'Photon': 22,
                'Pionpm': 211,
               # 'Klong': 130,
              #  'Kpm': 321,
               }
     
    masks_id_p = {'Pion0': 111,
                'Photon': 22,
                'Pionpm': 112,  
               }
    
    for part,pmask in masks_id.items():
        part_mask1p = (abs(target_kaon_dau1.pdgID)==pmask)
        part_mask2p = (abs(target_kaon_dau2.pdgID)==pmask)
        part_mask1k = (abs(target_kaon_dau1.mompdgID)== 310)
        part_mask2k = (abs(target_kaon_dau2.mompdgID)== 310)
        hists["Target_epz1_hist"].fill(Ptype=part,      
                                 e1=flat(target_kaon_dau1[part_mask1p & part_mask1k].e),
                                 pz1=flat(target_kaon_dau1[part_mask1p & part_mask1k].pz),
                                 theta1=flat(target_kaon_dau1[part_mask1p & part_mask1k].theta*(180/math.pi)),
                                )
        hists["Target_epz2_hist"].fill(Ptype=part,      
                                 e2=flat(target_kaon_dau2[part_mask2p & part_mask2k].e),
                                 pz2=flat(target_kaon_dau2[part_mask2p & part_mask2k].pz),
                                 theta2=flat(target_kaon_dau2[part_mask2p & part_mask2k].theta*(180/math.pi)),
                                )

In [6]:
nchunk = 0
for chunk in ldmx_dict:
    # I usually test my code in one chunck only
    #if nchunk >= 1: break
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

process 1
process 2
process 3
process 4
process 5
process 6
process 7
process 8
process 9
process 10
process 11
process 12
process 13
process 14
process 15
process 16
process 17
process 18
process 19
process 20
process 21
process 22
process 23
process 24
process 25


In [1]:
print(hists["Target_epz1_hist"])
fig, ax = plt.subplots(1,3,figsize=(24,6), constrained_layout=True)
hist.plot1d(hists["Target_epz1_hist"].sum('e1', 'theta1'),ax=ax[0],clear=False);
hist.plot1d(hists["Target_epz2_hist"].sum('e2', 'theta2'),ax=ax[0],clear=False);

hist.plot1d(hists["Target_epz1_hist"].sum('pz1', 'theta1'),ax=ax[1],clear=False);
hist.plot1d(hists["Target_epz2_hist"].sum('pz2', 'theta2'),ax=ax[1],clear=False);

hist.plot1d(hists["Target_epz1_hist"].sum('pz1', 'e1'),ax=ax[2],clear=False);
hist.plot1d(hists["Target_epz2_hist"].sum('pz2', 'e2'),ax=ax[2],clear=False);



#fig, ax = plt.subplots(1,3, figsize=(24,6), constrained_layout=True)
#hist.plot2d(hists["Target_kaon_ke"].integrate('Ktype', ['Kpm']), xaxis='k', ax=ax[0],patch_opts=POPTS);
#ax[0].set_title("$K^{\pm}$")

#hist.plot2d(hists["Target_kaon_ke"].integrate('Ktype', ['Klong']), xaxis='k', ax=ax[1],patch_opts=POPTS);
#ax[1].set_title("$K_l$")

#hist.plot2d(hists["Target_kaon_ke"].integrate('Ktype', ['Kshort']), xaxis='k', ax=ax,patch_opts=POPTS);
#ax.set_title("$K_s$")

NameError: name 'hists' is not defined